In [2]:
n = 8
eta = 11
q = n*eta+1
assert is_prime(q)
print q

89


In [3]:
Zq = IntegerModRing(q)
ZqX.<X> = PolynomialRing(Zq)
Pmod = X^n + 1
Rq = QuotientRing(ZqX,Pmod)
print(Pmod)
print(Rq)
factor(X^n+1)

X^8 + 1
Univariate Quotient Polynomial Ring in Xbar over Ring of integers modulo 89 with modulus X^8 + 1


(X^2 + 12) * (X^2 + 37) * (X^2 + 52) * (X^2 + 77)

In [3]:
w = Zq.multiplicative_generator()
a = w^(eta)
rrs = sorted([ ZZ(a^i)  for  i in range(n)])
assert(len(list(set(rrs)))==n)
assert(list(set([Zq(x)^(n) for x in rrs]))==[Zq(1)])
rrs = list(set(rrs))
print rrs
#a = Zq(rrs[1])
print a

[1, 34, 37, 12, 77, 52, 55, 88]
37


In [4]:
def revbits(i,nbits):
    bb = ZZ(i).digits(2)
    bb = bb+[ZZ(0)]*(nbits-len(bb))
    bb.reverse()
    return ZZ(bb,2)

def NTT(f):
    c = f.coefficients()
    c = c+[Zq(0)]*(n-len(c))
    k = 1
    l = n//2
    while l >= 2:
        s = 0
        while s < n:
            j = s
            while j < s + l:
                t = a^revbits(k,2)*c[j+l]
                c[j+l] = c[j] - t
                c[j] = c[j] + t
                j = j + 1
            k = k + 1
            s = j + l
        l = l//2
    return c

def NTTinv(c):
    k = 0
    l = 2
    while l <= n//2:
        s = 0
        while s < n:
            j = s
            while j < s + l:
                t = c[j]
                c[j] = t + c[j + l]
                c[j + l] = t - c[j + l]
                c[j + l] = Zq(ZZ(a^(revbits(k,2)+1)).inverse_mod(q)) * c[j+l]
                j = j + 1
            k = k + 1
            s = j + l
        l = l*2
    j = 0
    while j < n:
        c[j] = c[j] *ZZ(n//2).inverse_mod(q)
        j = j + 1
    return c

def polymul(c10,c11,c20,c21,i):
    ee = a^(2*revbits(i,2)+1)
    print(X^2-ee)
    r0  = c11 * c21
    r0  = r0 * ee
    r0 += c10 * c20
    r1  = c10 * c21
    r1 += c11 * c20
    return [r0,r1]


In [5]:
f = Rq.random_element().lift()
print f.coefficients()
ftt = NTT(f)
print(ftt)
fp = NTTinv(ftt)
print(fp)
print(ZqX(fp)==f)

[53, 46, 48, 17, 68, 61, 53, 76]
[60, 12, 42, 45, 11, 9, 10, 29]
[53, 46, 48, 17, 68, 61, 53, 76]
True


In [6]:
fs = map(lambda x : x[0],list(factor(Pmod)))
print fs
fs = [ X^2 - a^(i*2+1) for i in range(n//2)]
print fs

def NTTfake(f):
    cs = []
    pp = [ ((QuotientRing(ZqX,ff))(f)).lift() for ff in fs]
    for i in range(n//2):
        cs+=pp[i].coefficients()+[Zq(0)]*(2-len(pp[i].coefficients()))
    return cs

def NTTinvfake(v):
    ff = []
    i = 0
    while i < n//2:
        c0 = v[2*i]
        c1 = v[2*i+1]
        ff.append(c1*X+c0)
        i = i + 1
    return ZqX(CRT(ff,fs)).coefficients()

def polymul_fake(c10,c11,c20,c21,i):
    RR = QuotientRing(ZqX,fs[revbits(i,2)])
    print(fs[revbits(i,2)])
    f1f2 = (RR(c11*X+c10)*RR(c21*X+c20)).lift().coefficients()
    return f1f2+[Zq(0)]*(2-len(f1f2))

print NTTfake(f)
print NTTinvfake(NTTfake(f))

[X^2 + 12, X^2 + 37, X^2 + 52, X^2 + 77]
[X^2 + 52, X^2 + 77, X^2 + 37, X^2 + 12]
[60, 12, 11, 9, 42, 45, 10, 29]
[53, 46, 48, 17, 68, 61, 53, 76]


In [7]:
f1 = Rq.random_element().lift()
f2 = Rq.random_element().lift()
nf1 = NTT(f1)
nf2 = NTT(f2)
nf1f2 = []
i = 0
while i < n//2:
    c10 = nf1[2*i]
    c11 = nf1[2*i+1]
    c20 = nf2[2*i]
    c21 = nf2[2*i+1]
    f1f2 = polymul(c10,c11,c20,c21,i)
    nf1f2 += f1f2
    i = i + 1
print nf1f2
f1f2 = NTTinv(nf1f2)
print(Rq(f1)*Rq(f2))
print(Rq(f1f2))

X^2 + 52
X^2 + 37
X^2 + 77
X^2 + 12
[36, 54, 71, 79, 33, 21, 36, 9]
84*Xbar^7 + 88*Xbar^6 + 59*Xbar^5 + 33*Xbar^4 + 53*Xbar^3 + 66*Xbar^2 + 63*Xbar + 44
84*Xbar^7 + 88*Xbar^6 + 59*Xbar^5 + 33*Xbar^4 + 53*Xbar^3 + 66*Xbar^2 + 63*Xbar + 44
